# Statistics with python: A/B test

A/B testing is a tool that allows us to test two or more different ideas against each other, and to see which one empirically performs better.      

Why is A/B testing important ?       
1) no guessing:       
we run this test in real world, we get to know which idea is better under the conditions that matters most.       
2) provide accurate answer quickly.       
3) allows to rapidly iterate on ideas.       
4) one of the only statistiaclly sound ways to establish casual relationships.      

**A/B testing**      
A/B testing works by exposing unique randomly assigned groups of users to each of the ideas you want to test. Then we can observe these users, and by measuring how they behave, untangle the impact of each of your ideas and determine which is the best.      

If we have users and ideas, then chances are we can run an A/B test.       
Before performing A/B testing, we must first understand what is worth testing and optimizing for.

**Key Performance Indicator (KPIs)**            
Typically A/B tests are run to improve KPIs.       
These are the metrics that are most important to the business or organisation.       
For drug company: likelihood of a side-effect etc,     
For mobile game company: revenue, play time per user etc.      

Identifying KPIs requires domain knowledge + experience + EDA.       



## 1. Key Performance Indicator (KPIs)

### 1.1 Identifying and understanding KPIs

Example: Meditation App      
A mobile app that offers meditation services for a paid subscription as well as one-off in-app purchases.       

We are motivated to maintain a strong free-trial to paying user conversion rate. (KPIs)       

Dataset 1: User demographics      
tied to a unique user ID number.

In [3]:
import pandas as pd

In [ ]:
df_customer_data = pd.read_csv("../python_basics/data/user_demographics_v1.csv")

In [4]:
df_customer_data.head()

,uid,reg_date,device,gender,country,age
0,54030035.0,2017-06-29T00:00:00Z,and,M,USA,19
1,72574201.0,2018-03-05T00:00:00Z,iOS,F,TUR,22
2,64187558.0,2016-02-07T00:00:00Z,iOS,M,USA,16
3,92513925.0,2017-05-25T00:00:00Z,and,M,BRA,41
4,99231338.0,2017-03-26T00:00:00Z,iOS,M,FRA,59


Dataset 2: User actions        
Containing the data trial period and date of puchase and price paid in cent.

In [5]:
df_app_purchases = pd.read_csv("../python_basics/data/purchase_data_v1.csv")

In [6]:
df_app_purchases.head()

,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599
3,2017-09-26,91591874,sku_two_299,299
4,2017-12-01,91591874,sku_four_599,599


KPI: conversion rate       
One question in defining our KPI is over what interval should we consider the conversion rate. The conversion immediately after lapse, 1 week or 1 month after.       

One way to decide this is to see the generalisability of these statistics across different demographic groups.      
Stability in this way is desired